In [2]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline

Using Sagemaker SDK get your AWS account region, execution role and the default S3 bucket. Push the training and validation set along with the metadata files into this default S3 bucket.

In [2]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

session = sagemaker.Session()

bucket= session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-619836274583
AWS Region: us-east-1
RoleArn: arn:aws:iam::619836274583:role/service-role/AmazonSageMaker-ExecutionRole-20230220T203798


In AWS SageMaker, you can take advantage of prebuilt Docker images that support deep learning frameworks and other dependencies required for training and inference. We can leverage the Sagemaker Python SDK to utilize the latest machine learning or deep learning models. In the below code I have used the image_uris function to retrieve the latest image-classification image. The model artifacts would be stored in the location specified in the S3 path. Learn more about the pre built models here https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html.

In [3]:
# Use the image_uris function to retrieve the latest 'image-classification' image 
image_classification_model_image = sagemaker.image_uris.retrieve('image-classification',region,version='latest')
s3_path = f"s3://{bucket}/weather_classification_model"

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


Use the Sagemaker estimator to run the training job. This function requires the current Sagemaker execution role and session variables along with the other necessary parameters. There is a parameter called instance _count and it is set to 1. If you have millions of images for training, then you can increase the instance count proportionately and Sagemaker would distribute the training load across all the instances, and we don’t have to manage these instances.  You can find more about Sagemaker estimator here at https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-adapt-your-own-private-registry-estimator.html. 

In [4]:
image_classification_model = sagemaker.estimator.Estimator(
    image_uri=image_classification_model_image,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    volume_size=50,
    max_run=360000,
    input_mode="File",
    role=role,
    output_path=s3_path,
    sagemaker_session=session
)

We have the option of tuning the hyperparameters for our image classification model. Here two important parameters to be noted is use_pretrained_model=1 and multi_label=1. The latter indicates multi image classification while the former helps us with transfer learning because we want to make predictions on a custom dataset. We take a pre trained model, remove the final layers and retrain with our custom images. Additionally, I have set the image shape, number of classes, total number of training samples, etc,. There are several hyperparameters that can be tuned. Find them out here https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html. Also make sure that the image shape is 224 x 224 x 3 because earlier I used 300 x 300 x 3 and this caused problems during model inference.

In [5]:
image_classification_model.set_hyperparameters(
    image_shape='3,224,224' ,
    num_classes=4, 
    num_layers=18,
    num_training_samples=894,
    use_pretrained_model=1,
    multi_label=1,
    learning_rate=0.001,
    epochs=5
)

AWS offers several image formats for training models, including the one used in this project. To learn more about this method and other available options, please refer to the documentation at https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html. When training the model, it's important to specify four channels for the InputDataConfig parameter: train, validation, train_lst, and validation_lst. These channel names should not be changed. Additionally, the content type for all four channels should be set to application/x-image. To properly point to your image data, you can specify the main S3 path to the images in the s3_data parameter of the training input. The relative path or file name for each image should be included in the metadata file (lst file). This file should have three columns: the index of the image within the folder, the label of the image, and the relative path of the image.

In [6]:
from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput
model_inputs = {
        "train": sagemaker.inputs.TrainingInput(
            s3_data=f"s3://{bucket}/weather_classification/train/",
            content_type="application/x-image"
        ),
        "validation": sagemaker.inputs.TrainingInput(
            s3_data=f"s3://{bucket}/weather_classification/validation/",
            content_type="application/x-image"
        ),
        "train_lst": sagemaker.inputs.TrainingInput(
            s3_data=f"s3://{bucket}/weather_classification/train.lst",
            content_type="application/x-image"
        ),
        "validation_lst": sagemaker.inputs.TrainingInput(
            s3_data=f"s3://{bucket}/weather_classification/validation.lst",
            content_type="application/x-image"
        )
}

In [7]:
image_classification_model.fit(model_inputs)

INFO:sagemaker:Creating training-job with name: image-classification-2023-03-14-17-35-38-045


2023-03-14 17:35:38 Starting - Starting the training job...
2023-03-14 17:36:07 Starting - Preparing the instances for training.........
2023-03-14 17:37:19 Downloading - Downloading input data...
2023-03-14 17:37:53 Training - Downloading the training image............
2023-03-14 17:39:49 Training - Training image download completed. Training in progress....Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Tue Mar 14 17:40:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                  

We have achieved a training set accuracy of 99% and validation set accuracy of 95% which is a good performance.

Now let's deploy your model on a single ml.m5.xlarge instance and also add the data capture object created.

In [9]:
from sagemaker.serializers import IdentitySerializer

deployment = image_classification_model.deploy(
    initial_instance_count=1, instance_type='ml.m5.xlarge',
    serializer=IdentitySerializer(content_type="application/x-image")
    )

endpoint = deployment.endpoint_name
print(endpoint)

INFO:sagemaker:Creating model with name: image-classification-2023-03-14-17-44-15-030
INFO:sagemaker:Creating endpoint-config with name image-classification-2023-03-14-17-44-15-030
INFO:sagemaker:Creating endpoint with name image-classification-2023-03-14-17-44-15-030


---------!image-classification-2023-03-14-17-44-15-030


The below function randomly selects an object from test folder and stores it locally in a tmp folder for making predictions.

In [22]:
from sagemaker.serializers import IdentitySerializer
import boto3
import random
import os

s3_resource = boto3.resource('s3')
objects = s3_resource.Bucket(bucket).objects.filter(Prefix="weather_classification/test")
obj = random.choice([x.key for x in objects])
print('File selected ',obj)

s3_client = boto3.client('s3')

# with open(file_path, 'wb') as f:
s3_client.download_file(bucket, obj, 'tmp/image.png')
    

File selected  weather_classification/test/rain205.png


Create the predictor object for making predictions. You should pass the name of the endpoint and the session variable as parameters. IdentitySerializer helps in serializing the input for the inference endpoint and here we will specify the file type. We would have already specified the serializer in the deployment function. Both ways are acceptable.

The predictor will return all class probabilites as the output. Class label for the input image corresponds to the index of the highest probability value in the list.

In [40]:
predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint,
                                         sagemaker_session=session)

predictor.serializer = IdentitySerializer("image/png")

with open("tmp/image.png", "rb") as f:
    image = f.read()

    
result = predictor.predict(image)
print(result.decode('utf-8'))

[6.237225846916772e-08, 0.9989890456199646, 0.00011317242024233565, 0.0265173502266407]


There is an alternate way for getting predictions and this method would be handy while deploying the model in lambda function. We can make predictions by the invoke_endpoint method of the runtime object.

In [42]:
runtime= boto3.client('runtime.sagemaker')

response = runtime.invoke_endpoint(EndpointName=endpoint,
                                       ContentType='image/png',
                                       Body=image)

result = json.loads(response['Body'].read().decode())
    
print(result)

[6.237225846916772e-08, 0.9989890456199646, 0.00011317242024233565, 0.0265173502266407]


In [43]:
classes = ["cloudy", "rain", "shine", "sunrise"]
for i, val in enumerate(classes):
    print(classes[i], round(result[i]*100,2), end="% ")

cloudy 0.0% rain 99.9% shine 0.01% sunrise 2.65% 

If you are not using the endpoint please delete it. Because if left undeleted it would keep consuming resources.